<a href="https://colab.research.google.com/github/jihyunjeongme/data-analysis-kaggle/blob/master/%5BKaggle_2019_3rd_ML_month_with_KaKR%5D_%EC%9E%90%EB%8F%99%EC%B0%A8_%EC%9D%B4%EB%AF%B8%EC%A7%80_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%85%8B%EC%9D%84_%EC%9D%B4%EC%9A%A9%ED%95%9C_%EC%9E%90%EB%8F%99%EC%B0%A8_%EC%B0%A8%EC%A2%85%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# 필사 출처:
# https://www.kaggle.com/yangsaewon/pytorch-baseline-updated-7-10
# https://www.kaggle.com/fulrose/3rd-ml-month-car-model-classification-baseline

In [1]:
# 파이토치 사용하기
# 참고: https://medium.com/hyunjulie/구글-colab-에서-pytorch-사용하기-e065902d4a91

!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl
!pip3 install torchvision

     |████████████████████████████████| 592.3MB 1.1MB/s 
ERROR: torchvision 0.3.0 has requirement torch>=1.1.0, but you'll have torch 0.3.0.post4 which is incompatible.
ERROR: fastai 1.0.55 has requirement torch>=1.0.0, but you'll have torch 0.3.0.post4 which is incompatible.
  Found existing installation: torch 1.1.0
    Uninstalling torch-1.1.0:
      Successfully uninstalled torch-1.1.0
     |████████████████████████████████| 748.9MB 19kB/s 
  Found existing installation: torch 0.3.0.post4
    Uninstalling torch-0.3.0.post4:
      Successfully uninstalled torch-0.3.0.post4


In [2]:
import gc
import os
import warnings
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

from keras import backend as K
warnings.filterwarnings(action='ignore')

K.image_data_format()

Using TensorFlow backend.


'channels_last'

In [3]:
# test, traing set 데이터 불러오기

from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
cd /content/gdrive/My Drive/Colab Notebooks/dataanalysis/kaggle/2019_3rd_image_car_KaKR

/content/gdrive/My Drive/Colab Notebooks/dataanalysis/kaggle/2019_3rd_image_car_KaKR


## Preparation for the Data

### 파일 세부 설명
아래는 각 파일 별 세부 설명 입니다.

- train.csv - Train 셋의 이미지 파일명, 바운딩박스 좌표, 차종정보
- test.csv - Test 셋의 이미지 파일명, 바운딩박스 좌표, 차종정보
- submission.csv - Test셋과 대응되는 제출 파일
- class.csv - 데이터 셋의 class 컬럼과 대응되는 차종의 레이블
- train.zip - Train 이미지 파일
- test.zip - Test 이미지 파일



In [21]:
DATA_PATH = './'
os.listdir(DATA_PATH)

['sample_submission.csv',
 'test.zip',
 'test.csv',
 'train.zip',
 'train.csv',
 'class.csv',
 '2019-3rd-ml-month-with-kakr.zip',
 '2019-3rd-ml-month-with-kakr']

In [0]:
# 이미지 폴더 경로
TRAIN_IMG_PATH = os.path.join(DATA_PATH, 'train')
TEST_IMG_PATH = os.path.join(DATA_PATH, 'test')

# CSV 파일 경로
df_train = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))
df_test = pd.read_csv(os.path.join(DATA_PATH, 'test.csv'))
df_class = pd.read_csv(os.path.join(DATA_PATH, 'class.csv'))

# **Data Exploration**  
---
- 실제 데이터가 Description과 일치하는지, 데이터는 어떻게 구성되어 있고 크랠스 별로 어떤 분포를 가지고 있는지 등 데이터에 전반적으로 살펴봄


---
## Check Data
### 컬럼 세부 설명
각 컬럼 별 세부 설명 입니다.

- img_file - 데이터 셋의 각 로우와 연결되는 이미지 파일 이름
- bbox_x1 - 바운딩 박스 x1 좌표 (좌상단 x)
- bbox_y1 - 바운딩 박스 y1 좌표 (좌상단 y)
- bbox_x2 - 바운딩 박스 x2 좌표 (우하단 x)
- bbox_y2 - 바운딩 박스 y2 좌표 (우하단 y)
- class - 예측하려는 차종(Target)
- id - 각 데이터 셋에 기입 되어 있는 클래스 id
- name - 클래스 id에 대응되는 실제 차종 레이블

In [9]:
df_train.head()

,img_file,bbox_x1,bbox_y1,bbox_x2,bbox_y2,class
0,train_00001.jpg,1,80,641,461,108
1,train_00002.jpg,57,53,293,236,71
2,train_00003.jpg,35,42,227,160,76
3,train_00004.jpg,10,21,254,165,188
4,train_00005.jpg,1,67,772,573,44


In [10]:
df_test.head()

,img_file,bbox_x1,bbox_y1,bbox_x2,bbox_y2
0,test_00001.jpg,199,184,1116,809
1,test_00002.jpg,55,61,480,257
2,test_00003.jpg,42,123,602,413
3,test_00004.jpg,13,8,619,393
4,test_00005.jpg,8,9,209,93


In [29]:
# 데이터 누락 체크

if set(list(df_train.img_file)) == set(os.listdir(TRAIN_IMG_PATH)):
  print("Train file 누락 없음")
else :
  print("Train file 누락")
  
if set(list(df_test.img_file)) == set(os.listdir(TEST_IMG_PATH)) :
    print("Test file 누락 없음!")
else : 
    print("Test file 누락")

Train file 누락


FileNotFoundError: ignored